# Pipe 3

This notebook contains scripts for calculating kinematic data.

Input files must be `.csv` files generated by `rtklib-pos-to-csv.ipynb`.
Its headers are defined in the code below.

Outputs another `.csv` file _with the **different** headers_.
Those new headers are also defined in the code below.

In [1]:
import csv
import datetime as dt
import math
import pandas as pd
from geopy.distance import geodesic
from constants import *
from scipy.interpolate import Akima1DInterpolator

In [2]:
INPUT_FILEPATH = "/home/delaynie/Documents/rtklib-workspace/cmubuggy-datalogger/data/position-csv/"
INPUT_FILENAMES = ["output1.csv"] #, "output2.csv", "output3.csv"]
OUTPUT_FILEPATH = "/home/delaynie/Documents/rtklib-workspace/cmubuggy-datalogger/data/calc-csv/"

In [3]:
MIN_SATELLITES = 2

In [4]:
TIMESTAMP_FORMAT = "%Y/%m/%d %H:%M:%S.%f"

# TODO make this compatible with several levels of precesion
# converts timestamp string to UTC value (float)
def timestampToValue(ts: str):
    return dt.datetime.strptime(ts, TIMESTAMP_FORMAT).timestamp()

In [5]:
def getMagnitude(vector):
    return math.sqrt(math.pow(vector[0], 2) + math.pow(vector[1], 2))

In [6]:
def calc(inputPath, outputPath):
    input_df = pd.read_csv(INPUT_FILEPATH+INPUT_FILENAMES[0])

    interpolator = Akima1DInterpolator(input_df.loc[:, "timestamp"].map(timestampToValue), input_df.loc[:, ["latitude", "longitude"]])
    velocities = interpolator(input_df.loc[:, "timestamp"].map(timestampToValue), nu=1)
    accelerations = interpolator(input_df.loc[:, "timestamp"].map(timestampToValue), nu=2)

    output_df = pd.DataFrame(columns=CALC_CSV_HEADER)

    output_df["timestamp"] = input_df.loc[:, "timestamp"].map(timestampToValue)
    output_df["timestamp_str"] = input_df["timestamp"]

    output_df["fix_type"] = input_df["fix-type"]

    output_df["latitude"] = input_df["latitude"]
    output_df["longitude"] = input_df["longitude"]
    output_df["altitude"] = input_df["altitude"]

    output_df["velocity"] = velocities.tolist()
    output_df["speed"] = [getMagnitude(vector) for vector in velocities]

    output_df["acceleration_vector"] = accelerations.tolist()
    output_df["acceleration_magnitude"] = [getMagnitude(vector) for vector in accelerations]

    output_df.to_csv(outputPath)

In [7]:
for name in INPUT_FILENAMES:
    calc(INPUT_FILEPATH+name, OUTPUT_FILEPATH+name)